In [ ]:
!pip install pyarabic
# !pip install keras_preprocessing

In [ ]:
# from google.colab import drive
# import os
# import sys

# drive.mount('/content/drive')
# sys.path.append('/content/drive/MyDrive/Grammer_Correction/')

# os.chdir('/content/drive/MyDrive/Grammer_Correction/')

In [ ]:
import random
from enum import Enum
import re
import numpy as np
from pyarabic.araby import separate, tokenize, is_arabicrange, strip_tashkeel, strip_tatweel
# import tensorflow as tf
# from keras_preprocessing.sequence import pad_sequences
# from keras.models import Sequential,load_model, Model
# from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, CategoryEncoding, Bidirectional, Input, Concatenate, Dropout, TimeDistributed, RepeatVector, Flatten
# from keras.initializers import glorot_normal
# from keras.losses import SparseCategoricalCrossentropy, CategoricalFocalCrossentropy
# from keras.metrics import SparseCategoricalAccuracy, F1Score
# from keras.utils import plot_model,to_categorical
from gensim.models import Word2Vec, FastText,KeyedVectors
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
# from torchtext.data import Field, BucketIterator
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

file_path_alignments = "/kaggle/input/arabic-gec-datasets/datasets/alignments/"
file_path_datasets = "/kaggle/input/arabic-gec-datasets/datasets/ged_datasets/"

# from scripts_ged.ged_scripts.ged_preprocessor import *

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [ ]:
def read_arabic_for_word2vec(input_filepath):
    all_sentences = []
    arabic_list = []
    with open(input_filepath, 'r', encoding='utf-8') as input_file:
        for line in input_file:
            if line.strip() == ".":
              arabic_list.append(line.strip())
              all_sentences.append(arabic_list)
              arabic_list = []
            else:
              arabic_list.append(line.strip())
    return all_sentences


In [ ]:
def read_alignments(input_file_name):
  space_token = '<SPACE>'
  source_list = []
  target_list = []

  with open(input_file_name, 'r', encoding='utf-8') as file:
      count = 1  # Initialize count to 0 to correctly skip the header
      for line in file:
          string = line
          line = line.split('\t')
          if count == 1:  # Skip the first line (header)
              count += 1
              continue
          if len(line) == 2:
            if line[1].strip() == '':
              source_list.append(line[0].strip())
              target_list.append(space_token)
            # print(line)
            elif string.startswith('\t'):
              # print(string)
              # print(line)
              # print(line)
              source_list.append(space_token)
              target_list.append(line[1].strip())
            else:
              # print(string)
              # print(line)
              source_list.append(line[0].strip())
              target_list.append(line[1].strip())
          # print(source_list)
          count+=1
          # print(count)

  return source_list, target_list


In [ ]:
def create_windows(source_list,target_list,window_size):
  source_windows = []
  target_windows = []
  assert len(source_list) == len(target_list)
  for i in range(0,len(source_list),window_size):
    source_window = source_list[i:i+window_size] if i+window_size <= len(source_list) else source_list[i:]
    target_window = target_list[i:i+window_size] if i+window_size <= len(target_list) else target_list[i:]
    source_windows.append(['<SOS>'] + source_window + ['<EOS>'])
    target_windows.append(['<SOS>'] + target_window + ['<EOS>'])
  return source_windows, target_windows


In [ ]:
def convert_to_indicies(source_list, word_to_index):
  source_indicies = []
  for source in source_list:
    source_indicies.append([word_to_index.get(word,word_to_index['<UNK>']) for word in source])
  return source_indicies


In [ ]:
def create_word_to_index(token_lists):
    # Initialize the dictionary with special tokens
    word_to_index = {'<SOS>': 0, '<EOS>': 1}
    
    # Start the index from 2 since 0 and 1 are already used
    index = 2
    
    # Iterate through the list of lists
    for token_list in token_lists:
        for token in token_list:
            # Add the token to the dictionary if it's not already present
            if token not in word_to_index:
                word_to_index[token] = index
                index += 1
                
    if '<UNK>' not in word_to_index:
        word_to_index['<UNK>'] = index
        index+=1
    if '<PAD>' not in word_to_index:
        word_to_index['<PAD>'] =index
        index+=1
        
    return word_to_index

In [ ]:
import gc
source_list1, target_list1 = read_alignments(file_path_alignments + 'qalb14/qalb14_train.txt')
source_list1, target_list1 = create_windows(source_list1,target_list1,window_size = 5)
print(len(source_list1))

source_list2, target_list2 = read_alignments(file_path_alignments + 'qalb15/qalb15_train.txt')
source_list2, target_list2 = create_windows(source_list2,target_list2,window_size = 5)
print(len(source_list2))

source_list3, target_list3 = read_alignments(file_path_alignments + 'zaebuc/zaebuc_train.txt')
source_list3, target_list3 = create_windows(source_list3,target_list3,window_size = 5)
print(len(source_list3))

source_list = []
target_list = []

source_list += (source_list1 + source_list2 + source_list3)

target_list += (target_list1 + target_list2 + target_list3)

del source_list1 , source_list2, source_list3, target_list1, target_list2, target_list3
gc.collect()
print(len(source_list))




In [ ]:
# with open('output.txt','w') as f:
#     for i,j in zip(source_list, target_list):
#         f.write(i + '\t' + j + '\n')

        

In [ ]:
print(source_list[-7])
print(target_list[-7])
print(len(target_list))

## Word2vec Finetuning

In [ ]:
arabic_list = read_arabic_for_word2vec(file_path_datasets + "splitted_qalb14/train.txt")
arabic_list2 = read_arabic_for_word2vec(file_path_datasets + "splitted_qalb15/qalb15_train.txt")
arabic_list3 = read_arabic_for_word2vec(file_path_datasets + "splitted_zaebuc/zaebuc_train.txt")
arabic_list4 = read_arabic_for_word2vec(file_path_datasets  + "splitted_qalb14/dev.txt")
arabic_list5 = read_arabic_for_word2vec(file_path_datasets  + "splitted_qalb14/test.txt")
arabic_list6 = read_arabic_for_word2vec(file_path_datasets  + "splitted_zaebuc/dev_train.txt")
arabic_list.extend(arabic_list2)
arabic_list.extend(arabic_list3)
arabic_list.extend(arabic_list4)
arabic_list.extend(arabic_list5)
arabic_list.extend(arabic_list6)
# add <SOS> and <EOS> tokens at beginning and end of each sentence in arabic_list where each sentence is a list of tokens
for i in range(len(arabic_list)):
  arabic_list[i] = ['<SOS>'] + arabic_list[i] + ['<EOS>']

arabic_list.extend(target_list)


word2vec_model = Word2Vec(sentences=arabic_list, vector_size=256, window=5, min_count=0, workers=4, sg=1)

In [ ]:
print(arabic_list[0])

## Glove model vectors


In [ ]:
# load glove model
# glove_model = KeyedVectors.load('./models/glove_word2vec_model.kv')


## Embedding matrix creation


In [ ]:
all_tokens = source_list + target_list
word_to_index = create_word_to_index(all_tokens)
index_to_word = {idx: word for word, idx in word_to_index.items()}
vocab_size = len(word_to_index)
print(vocab_size)
word_to_index['المظلم']

In [ ]:
combined_vocab = set(word_to_index.keys())
print(len(combined_vocab))

In [ ]:
source_to_index = create_word_to_index(source_list)
print(len(source_to_index))
source_vocab_size = len(source_to_index)
print(source_to_index['المضلم'])

In [ ]:
target_to_index = create_word_to_index(target_list)
print(len(target_to_index))
target_vocab_size = len(target_to_index)
print(target_to_index['المظلم'])
index_to_word_target = {idx: word for word, idx in target_to_index.items()}

In [ ]:
word2vec_vocab = set(word2vec_model.wv.key_to_index.keys())
# # glove_vocab = set(glove_model.key_to_index.keys())
# combined_vocab = word2vec_vocab.copy() # .union(glove_vocab)
# # print(len(glove_vocab))
# print(len(word2vec_vocab))
# print(len(combined_vocab))
# combined_vocab.add('<UNK>')
# combined_vocab.add('<SPACE>')
# combined_vocab.add('<PAD>')
# print(len(combined_vocab))

In [ ]:
# word_to_index = {word: idx for idx, word in enumerate(combined_vocab)}
# index_to_word = {idx: word for word, idx in word_to_index.items()}
# index = word_to_index["<UNK>"]
# index_to_word[index]
# vocab_size = len(combined_vocab)

In [ ]:
import pickle

with open('source2index.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(source_to_index, file)
    
with open('target2index.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(target_to_index, file)

    
with open('index2word_target.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(index_to_word_target, file)
    
    

In [ ]:
source_vocab = set(source_to_index.keys())
print(len(source_vocab))

In [ ]:
target_vocab = set(target_to_index.keys())
print(len(target_vocab))

In [ ]:
def create_embedding_matrix(vocab, vocab_size, embedding_dim, word_to_index, word2vec_model,word2vec_vocab):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    unk = 0
    mean = np.mean(word2vec_model.wv.vectors, axis=0)
    for word in vocab:
        # if word in glove_vocab:
        #     embedding_matrix[word_to_index[word]] = glove_model[word]
        if word == '<SPACE>':
            embedding_matrix[word_to_index[word]] = np.zeros(embedding_dim)
        elif word == '<PAD>':
            embedding_matrix[word_to_index[word]] = np.ones(embedding_dim)
        elif word in word2vec_vocab:
            embedding_matrix[word_to_index[word]] = word2vec_model.wv[word]
        else:
            # <UNK> token
                unk+=1
                embedding_matrix[word_to_index[word]] = mean

    print(unk)
    return embedding_matrix
        

In [ ]:
source_embedding_matrix = create_embedding_matrix(source_vocab,source_vocab_size, 256, source_to_index, word2vec_model, word2vec_vocab)
target_embedding_matrix = create_embedding_matrix(target_vocab,target_vocab_size, 256, target_to_index, word2vec_model, word2vec_vocab)
embedding_matrix = create_embedding_matrix(combined_vocab,vocab_size, 256, word_to_index, word2vec_model, word2vec_vocab)


In [ ]:
source_embedding_matrix.shape

In [ ]:
target_embedding_matrix.shape

In [ ]:
embedding_matrix.shape

## Encoder Definition


In [ ]:
class EncoderLSTM(nn.Module):

  def __init__(self,input_size, embedding_dim, embedding_matrix , hidden_size, num_layers, dropout_p=0.1):
    super(EncoderLSTM,self).__init__()
    self.embedding = nn.Embedding(input_size,embedding_dim)
    self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))

#     self.embedding.weight.requires_grad = False # to prevent trainable embeddings (subject to trial)

    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout_p) # (number of layers, batch_size, hidden_size)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self,input):

     # input = [max_seq_length,batch size]
    #  print('input to encoder shape ', input.shape)


     embedded = self.dropout(self.embedding(input))
     # embedded = [max_seq_length, batch size, embedding_dim]
    #  print("encoder input to lstm shape", embedded.shape)

     output, (hidden, cell) = self.lstm(embedded)
     return hidden, cell


## Decoder Definition

In [ ]:

class DecoderLSTM(nn.Module):
    def __init__(self, output_size,embedding_dim, embedding_matrix, hidden_size, num_layers, dropout_p=0.1):
        super(DecoderLSTM, self).__init__()
        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))

        # If you want to freeze the embedding layer (no training)
#         self.embedding.weight.requires_grad = False

        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout_p) # hidden size of decoder should be the same as the encoder
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0) # since input is of shape N (Batch_size)
        # shape of input: (1,Batch_size) --> one word per batch as the decoder outputs one word at a time
        # print("input to decoder shape ",input.shape)
        output = self.dropout(self.embedding(input))
        # shape of output: (1,Batch_size,embedding_dim)
        # print("output of embedding layer shape", output.shape)
        # print("hidden input to decoder shape ", hidden.shape)
        output, (hidden,cell) = self.lstm(output, (hidden,cell))
        # shape of output: (1,Batch_size,hidden_size)
        # print("output of lstm of decoder shape ", output.shape)
        output = self.out(output)
        # print("output of dense of decoder shape ", output.shape)
        # shape of output: (1,Batch_size,output_size(Vocab_size))

        output = output.squeeze(0)
        # print("output of lstm of decoder shape after squeezing ", output.shape)

        # shape of output: (Batch_size,output_size(Vocab_size))
        return output, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self,source,target,teacher_force_ratio = 0.5):

        batch_size = source.shape[1]  # source shape (seq_len, batch_size)
        # print("batch_size ",batch_size)
        target_len = target.shape[0]    # target shape (seq_len, batch_size)
        # print("target_length ",target_len)
        target_vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(target_len,batch_size,target_vocab_size).to(device)
        hidden, cell = self.encoder(source)
        # print("encoder output shape ", hidden.shape)
        # shape of hidden: (num_layers * num_directions, Batch_size, hidden_size)
        # shape of cell: (num_layers * num_directions, Batch_size, hidden_size)
        x = target[0]
        # print("target[0] ",x, x.shape)
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = output
            max_pred = output.argmax(1)
            x = target[t] if random.random() < teacher_force_ratio else max_pred
        return outputs
    
    def inference(self, source, max_len, start_token_id = target_to_index['<SOS>']):
        batch_size = source.shape[1]
        target_vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(device)
        hidden, cell = self.encoder(source)

        # Start token, assuming it's the first token in the target vocab
        x = torch.full((batch_size,), start_token_id, dtype=torch.long).to(device)
        
        for t in range(1, max_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = output
            max_pred = output.argmax(1)
            x = max_pred
        
        return outputs



## Data preperation

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, source, target,source_pad_index = source_to_index['<PAD>'],target_pad_index = target_to_index['<PAD>']):
        self.source = source
        self.target = target
        self.max_length = max(max(len(s) for s in source), max(len(t) for t in target))

        self.source = torch.tensor([s + [source_pad_index] * (self.max_length - len(s)) for s in source])
        self.target = torch.tensor([t + [target_pad_index] * (self.max_length - len(t)) for t in target])

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

In [ ]:
print(source_list[-1],target_list[-1])

In [ ]:
# print(word_to_index['<PAD>'])

In [ ]:
source_list = convert_to_indicies(source_list,source_to_index)
target_list = convert_to_indicies(target_list,target_to_index)


In [ ]:
print(source_list[-1],target_list[-1])

In [ ]:
batch_size = 5
mini_sentences = source_list[0: 8]
mini_labels = target_list[0: 8]
mini_dataset = Dataset(mini_sentences, mini_labels)
dummy_dataloader = torch.utils.data.DataLoader(mini_dataset, batch_size=5)
dg = iter(dummy_dataloader)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

In [ ]:
dataset = Dataset(source_list, target_list)

## Training Hyperparamaters

In [ ]:
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [ ]:
print(num_epochs)

## Model Hyperparamaters


In [ ]:
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_input_size = source_vocab_size
decoder_input_size = target_vocab_size
embedding_size = 256
hidden_size = 1024
num_layers = 2
dropout_p = 0.5
print(device)


In [ ]:
print(hidden_size)

In [ ]:
print(hidden_size)

In [ ]:
encoder = EncoderLSTM(
    encoder_input_size,
    embedding_size,
    source_embedding_matrix,
    hidden_size,
    num_layers,
    dropout_p
).to(device)

In [ ]:
decoder = DecoderLSTM(
    decoder_input_size,
    embedding_size,
    target_embedding_matrix,
    hidden_size,
    num_layers,
    dropout_p,
).to(device)

In [ ]:
model = Seq2Seq(encoder, decoder).to(device)
print(model)


## Training loop

In [ ]:
def eval_fun(training_example,model):
    lis = [word_to_index.get(word,source_to_index['<UNK>']) for word in training_example]
    tensor = torch.tensor(lis, dtype=torch.long)
    tensor = tensor.unsqueeze(1)
    tensor = tensor.to(device)
    output = model.inference(tensor,tensor.shape[0])
    output = output.reshape(-1, output.shape[2])
    output = output.argmax(dim=1)
    final = []
    for indx in output:
        final.append(index_to_word_target[indx.item()])
    
    return final
        
   

In [ ]:
def train(model, train_dataset, training_example, batch_size=10, epochs=5, learning_rate=0.01,model_name = "model"):
  """
  This function implements the training logic
  Inputs:
  - model: the model ot be trained
  - train_dataset: the training set of type Dataset
  - batch_size: integer represents the number of examples per step
  - epochs: integer represents the total number of epochs (full training pass)
  - learning_rate: the learning rate to be used by the optimizer
  """
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    for source, targets in tqdm(train_dataloader):
      source = source.transpose(0,1)
      targets = targets.transpose(0,1)
      source = source.to(device)
      targets = targets.to(device)
      # print(source.shape)
      # print(targets.shape)
      output = model(source,targets)
      # print("Output of Seq2seq ", output.shape)
      output = output[1:].reshape(-1, output.shape[2]) # to neglect start token
      # print("Output of Seq2seq after reshaping ", output.shape)
      # print("target shape ", targets.shape)
      targets = targets[1:].reshape(-1) # to neglect start token
      # print("target shape after reshaping ", targets.shape)

      batch_loss = criterion(output, targets)
      total_loss_train += batch_loss.item()
      # print("shape of argmax ", output.argmax(dim=1).shape)
      acc = (output.argmax(dim=1) == targets).sum().item()
      total_acc_train += acc

      optimizer.zero_grad()
      batch_loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
      optimizer.step()

    epoch_loss = total_loss_train / len(train_dataset)
    epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0][0]))
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')
    print("sentence evaluation",eval_fun(training_example,model))
    
  torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, model_name + '_checkpoint.pth')  

In [ ]:
test =  ['<SOS>', 'الشي', 'عن', 'برزة', 'وانا', 'ببرزة', '<EOS>']

In [ ]:
target_to_index['<SOS>']


In [ ]:
train(model,dataset,test,batch_size,num_epochs,learning_rate)

In [ ]:
def evaluate(model,test_dataset,batch_size):
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

    # GPU Configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    total_acc_eval = 0
    with torch.no_grad():
        for source, targets in tqdm(test_dataloader):
            source = source.transpose(0,1)
            targets = targets.transpose(0,1)
            # Move the test input to the device
            source = source.to(device)
            # Move the test label to the device
            targets = targets.to(device)

            # Perform the forward pass
            output = model.inference(source,source.shape[0])

            output = output[1:].reshape(-1, output.shape[2]) 
          
            targets = targets[1:].reshape(-1) 

            # print("shape of argmax ", output.argmax(dim=1).shape)
            acc = (output.argmax(dim=1) == targets).sum().item()
            total_acc_eval += acc
     
    
    acc = total_acc_eval / (len(test_dataset) * len(test_dataset[0][0]))
    print(f'Evaluation Accuracy: {acc}')


In [ ]:
print(eval_fun(test,model))

In [ ]:
torch.save(model, 'model.pickle')

In [ ]:
test_source, test_targets = read_alignments(file_path_alignments + 'qalb14/qalb14_train.txt')
test_source, test_targets = create_windows(test_source, test_targets, window_size = 5)
# test_source = convert_to_indicies(test_source, source_to_index)
# test_targets = convert_to_indicies(test_targets, target_to_index)
print(test_source[-4],test_targets[-4])

In [ ]:
test_dataset = Dataset(test_source, test_targets)

In [ ]:
evaluate(model,test_dataset,batch_size)

In [ ]:
import os 
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink 
FileLink(r'model_checkpoint.pth')